In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp numerframe

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import uuid
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Union, Tuple, Any, List

from numerblox.misc import AttrDict

## Overview: The NumerFrame

`NumerFrame` is a data structure that extends `pd.DataFrame` with functionality convenient for Numerai users. The main benefits include:
1. Automatically track features, targets, prediction and other columns + easily retrieve these data slices.
2. Other library functionality automatically recognizes era column (`era`, `friday_date` or `date`).
3. Integrations with other library components (i.e. `preprocessing`, `model`, `modelpipeline`, `postprocessing`, `evaluation` and `submission`) to create more solid inference pipelines and increase reliability.

Besides, all functionality of Pandas DataFrames is still available in the `NumerFrame`. You therefore don't have to create new pipelines to process your data when using `NumerFrame`.

We adopt the convention:
 1. All feature column names should start with `'feature'`.
 2. All target column names should start with `'target'`.
 3. All prediction column names should start with `'prediction'`.
 4. Data should contain an `'era'`, `'friday_date'` or `'date'` column, as is almost always the case with Numerai datasets.

Every column for which these conditions do not hold will be classified as an `'aux'` column.

In [ ]:
#| export
class NumerFrame(pd.DataFrame):
    """
    Data structure which extends Pandas DataFrames and
    allows for additional Numerai specific functionality.
    """
    _metadata = ["meta", "feature_cols", "target_cols",
                 "prediction_cols", "not_aux_cols", "aux_cols"]


    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.meta = AttrDict()
        self.__set_era_col()
        self.__init_meta_attrs()
        
    @property
    def _constructor(self):
        return NumerFrame

    def __init_meta_attrs(self):
        """ Dynamically track column groups. """
        self.feature_cols = [col for col in self.columns if str(col).startswith("feature")]
        self.target_cols = [col for col in self.columns if str(col).startswith("target")]
        self.prediction_cols = [
            col for col in self.columns if "prediction" in str(col)
        ]
        self.not_aux_cols = self.feature_cols + self.target_cols + self.prediction_cols
        self.aux_cols = [
            col for col in self.columns if col not in self.not_aux_cols
        ]

    def __set_era_col(self):
        """ Each NumerFrame should have an era column to benefit from all functionality. """
        if "era" in self.columns:
            self.meta.era_col = "era"
        elif "friday_date" in self.columns:
            self.meta.era_col = "friday_date"
        elif "date" in self.columns:
            self.meta.era_col = "date"
        else:
            self.meta.era_col = None

    def get_column_selection(self, cols: Union[str, list]):
        """ Return NumerFrame from selection of columns. """
        return self.loc[:, cols if isinstance(cols, list) else [cols]]

    @property
    def get_feature_data(self):
        """ All columns for which name starts with 'target'."""
        return self.get_column_selection(cols=self.feature_cols)

    @property
    def get_target_data(self):
        """ All columns for which name starts with 'target'."""
        return self.get_column_selection(cols=self.target_cols)

    @property
    def get_single_target_data(self):
        """ Column with name 'target' (Main Numerai target column). """
        return self.get_column_selection(cols=['target'])

    @property
    def get_prediction_data(self):
        """ All columns for which name starts with 'prediction'."""
        return self.get_column_selection(cols=self.prediction_cols)

    @property
    def get_aux_data(self):
        """ All columns that are not features, targets or predictions. """
        return self.get_column_selection(cols=self.aux_cols)

    @property
    def get_prediction_aux_data(self):
        """ All predictions columns and aux columns (for ensembling, etc.). """
        return self.get_column_selection(cols=self.prediction_cols+self.aux_cols)

    def get_pattern_data(self, pattern: str):
        """
        Get columns based on pattern (for example '_20' to get all 20-day Numerai targets).
        :param pattern: A 'like' pattern (pattern in column_name == True)
        """
        return self.filter(like=pattern)

    def get_feature_target_pair(self, multi_target=False) -> Tuple[Any, Any]:
        """
        Get split of feature and target columns.
        :param multi_target: Returns only 'target' column by default.
        Returns all target columns when set to True.
        """
        X = self.get_feature_data
        y = self.get_target_data if multi_target else self.get_single_target_data
        return X, y

    def get_era_batch(self, eras: List[Any],
                      convert_to_tf = False,
                      aemlp_batch = False,
                      features: list = None,
                      targets: list = None,
                      *args, **kwargs) -> tuple:
        """
        Get feature target pair batch of 1 or multiple eras. \n
        :param eras: Selection of era names that should be present in era_col. \n
        :param convert_to_tf: Convert to tf.Tensor. \n
        :param aemlp_batch: Specific target batch for autoencoder training. \n
        `y` output will contain three components: features, targets and targets. \n
        :param features: List of features to select. All by default \n
        :param targets: List of targets to select. All by default. \n
        *args, **kwargs are passed to initialization of Tensor.
        """
        valid_eras = []
        for era in eras:
            assert era in self[self.meta.era_col].unique(), f"Era '{era}' not found in era column ({self.meta.era_col})"
            valid_eras.append(era)
        features = features if features else self.feature_cols
        targets = targets if targets else self.target_cols
        X = self.loc[self[self.meta.era_col].isin(valid_eras)][features].values
        y = self.loc[self[self.meta.era_col].isin(valid_eras)][targets].values
        if aemlp_batch:
            y = [X.copy(), y.copy(), y.copy()]

        if convert_to_tf:
            import tensorflow as tf
            X = tf.convert_to_tensor(X, *args, **kwargs)
            if aemlp_batch:
                y = [tf.convert_to_tensor(i, *args, **kwargs) for i in y]
            else:
                y = tf.convert_to_tensor(y, *args, **kwargs)
        return X, y

`create_numerframe` automatically recognizes your data file format, loads it into a `NumerFrame` and allows for column selection before loading.

Support file formats are `.csv`, `.parquet`, `.pkl`, `.pickle`, `.xsl`, `.xslx`, `.xlsm`, `.xlsb`, `.odf`, `.ods` and `.odt`. If the file format for your use case is missing, feel free to create a Github issue or submit a pull request. See `README.md` for more information on contributing.

In [ ]:
#| export
def create_numerframe(file_path: str, columns: list = None, *args, **kwargs) -> NumerFrame:
    """
    Convenient function to initialize NumerFrame.
    Support most used file formats for Pandas DataFrames \n
    (.csv, .parquet, .xls, .pkl, etc.).
    For more details check https://pandas.pydata.org/docs/reference/io.html

    :param file_path: Relative or absolute path to data file. \n
    :param columns: Which columns to read (All by default). \n
    *args, **kwargs will be passed to Pandas loading function.
    """
    assert Path(file_path).is_file(), f"{file_path} does not point to file."
    suffix = Path(file_path).suffix
    if suffix in [".csv"]:
        df = pd.read_csv(file_path, usecols=columns, *args, **kwargs)
    elif suffix in [".parquet"]:
        df = pd.read_parquet(file_path, columns=columns, *args, **kwargs)
    elif suffix in [".xls", ".xlsx", ".xlsm", "xlsb", ".odf", ".ods", ".odt"]:
        df = pd.read_excel(file_path, usecols=columns, *args, **kwargs)
    elif suffix in ['.pkl', '.pickle']:
        df = pd.read_pickle(file_path, *args, **kwargs)
        df = df.loc[:, columns] if columns else df
    else:
        raise NotImplementedError(f"Suffix '{suffix}' is not supported.")
    num_frame = NumerFrame(df)
    return num_frame

## NumerFrame Usage

A `NumerFrame` object can be initialized from memory just like you would with a Pandas DataFrame.

### 1. Initialize from memory

In [ ]:
test_features = [f"feature_{l}" for l in "ABCDEFGHIK"]
id_col = [uuid.uuid4().hex for _ in range(100)]

# Random DataFrame
dataf = pd.DataFrame(np.random.uniform(size=(100, 10)), columns=test_features)
dataf["id"] = id_col
dataf[["target", "target_1", "target_2"]] = np.random.normal(size=(100, 3))
dataf["date"] = range(100)

In [ ]:
memory_dataf = NumerFrame(dataf)
assert memory_dataf.meta.era_col == "date"

In [ ]:
memory_dataf.head(2)

feature_A  feature_B  feature_C  feature_D  feature_E  feature_F  \
0   0.939419   0.248241   0.131653   0.078532   0.203165   0.704196   
1   0.422693   0.537752   0.137142   0.686566   0.582026   0.103345   

   feature_G  feature_H  feature_I  feature_K  \
0   0.152887   0.053677   0.890165   0.629224   
1   0.748019   0.186389   0.500554   0.943307   

                                 id    target  target_1  target_2  date  
0  35b61083cea044b280f3d33ebd55b420  1.630649 -1.932858  0.542668     0  
1  ce5d2abc49fd40849981a3694b96334f  0.811435 -0.841065  1.128461     1

The `meta` attribute will store which era column is used. This is used in NumerBlox processors to group computations by era where needed.

In [ ]:
memory_dataf.meta

{'era_col': 'date'}

### 2. Initialize from file path

You can also use the convenience function `create_numerframe` so `NumerFrame` can be easily initialized. Think of it as a dynamic `pd.read_csv`, `pd.read_parquet`, etc.

In [ ]:
num_dataf = create_numerframe("test_assets/mini_numerai_version_2_data.parquet",
                          )
assert num_dataf.meta.era_col == "era"

### 3. Example functionality

`.get_feature_data` will retrieve all columns where the column name starts with `feature`.

In [ ]:
num_dataf.get_feature_data.head(2)

feature_dichasial_hammier_spawner  \
id                                                    
n559bd06a8861222                               0.25   
n9d39dea58c9e3cf                               0.75   

                  feature_rheumy_epistemic_prancer  \
id                                                   
n559bd06a8861222                              0.75   
n9d39dea58c9e3cf                              0.50   

                  feature_pert_performative_hormuz  \
id                                                   
n559bd06a8861222                              0.25   
n9d39dea58c9e3cf                              0.75   

                  feature_hillier_unpitied_theobromine  \
id                                                       
n559bd06a8861222                                  0.75   
n9d39dea58c9e3cf                                  1.00   

                  feature_perigean_bewitching_thruster  \
id                                                       
n559bd06a8861222                                  0.25   
n9d39dea58c9e3cf                                  0.50   

                  feature_renegade_undomestic_milord  \
id                                                     
n559bd06a8861222                                0.50   
n9d39dea58c9e3cf                                0.25   

                  feature_koranic_rude_corf  \
id                                            
n559bd06a8861222                        1.0   
n9d39dea58c9e3cf                        0.5   

                  feature_demisable_expiring_millepede  \
id                                                       
n559bd06a8861222                                  0.25   
n9d39dea58c9e3cf                                  0.00   

                  feature_unscheduled_malignant_shingling  \
id                                                          
n559bd06a8861222                                     0.25   
n9d39dea58c9e3cf                                     1.00   

                  feature_clawed_unwept_adaptability  ...  \
id                                                    ...   
n559bd06a8861222                                0.75  ...   
n9d39dea58c9e3cf                                0.25  ...   

                  feature_unpruned_pedagoguish_inkblot  \
id                                                       
n559bd06a8861222                                  0.75   
n9d39dea58c9e3cf                                  1.00   

                  feature_forworn_hask_haet  \
id                                            
n559bd06a8861222                        0.0   
n9d39dea58c9e3cf                        1.0   

                  feature_drawable_exhortative_dispersant  \
id                                                          
n559bd06a8861222                                     1.00   
n9d39dea58c9e3cf                                     0.25   

                  feature_metabolic_minded_armorist  \
id                                                    
n559bd06a8861222                                0.0   
n9d39dea58c9e3cf                                0.5   

                  feature_investigatory_inerasable_circumvallation  \
id                                                                   
n559bd06a8861222                                               0.0   
n9d39dea58c9e3cf                                               0.0   

                  feature_centroclinal_incentive_lancelet  \
id                                                          
n559bd06a8861222                                     0.25   
n9d39dea58c9e3cf                                     0.25   

                  feature_unemotional_quietistic_chirper  \
id                                                         
n559bd06a8861222                                    0.00   
n9d39dea58c9e3cf                                    0.75   

                  feature_behaviorist_microbiological_farina  \
id                                       

`.get_target_data` retrieves all columns if the column name starts with `"target"`.

In [ ]:
num_dataf.get_target_data.head(2)

target  target_nomi_20  target_nomi_60  target_jerome_20  \
id                                                                           
n559bd06a8861222    0.25            0.25            0.50               0.0   
n9d39dea58c9e3cf    0.50            0.50            0.75               0.5   

                  target_jerome_60  target_janet_20  target_janet_60  \
id                                                                     
n559bd06a8861222              0.50              0.5              0.5   
n9d39dea58c9e3cf              0.75              0.5              0.5   

                  target_ben_20  target_ben_60  target_alan_20  ...  \
id                                                              ...   
n559bd06a8861222           0.25            0.5             0.5  ...   
n9d39dea58c9e3cf           0.50            0.5             0.5  ...   

                  target_paul_20  target_paul_60  target_george_20  \
id                                                                   
n559bd06a8861222             0.0            0.50              0.25   
n9d39dea58c9e3cf             0.5            0.75              0.50   

                  target_george_60  target_william_20  target_william_60  \
id                                                                         
n559bd06a8861222               0.5           0.000000           0.500000   
n9d39dea58c9e3cf               0.5           0.666667           0.666667   

                  target_arthur_20  target_arthur_60  target_thomas_20  \
id                                                                       
n559bd06a8861222          0.166667          0.500000          0.333333   
n9d39dea58c9e3cf          0.500000          0.666667          0.500000   

                  target_thomas_60  
id                                  
n559bd06a8861222          0.500000  
n9d39dea58c9e3cf          0.666667  

[2 rows x 21 columns]

`.get_single_target_data` only retrieves the column `"target"`.

In [ ]:
num_dataf.get_single_target_data.head(2)

target
id                      
n559bd06a8861222    0.25
n9d39dea58c9e3cf    0.50

`.get_pattern_data` allows you to get columns based on a certain pattern. In this example we retrieve all 20-day targets.

In [ ]:
num_dataf.get_pattern_data("_20").head(2)

target_nomi_20  target_jerome_20  target_janet_20  \
id                                                                    
n559bd06a8861222            0.25               0.0              0.5   
n9d39dea58c9e3cf            0.50               0.5              0.5   

                  target_ben_20  target_alan_20  target_paul_20  \
id                                                                
n559bd06a8861222           0.25             0.5             0.0   
n9d39dea58c9e3cf           0.50             0.5             0.5   

                  target_george_20  target_william_20  target_arthur_20  \
id                                                                        
n559bd06a8861222              0.25           0.000000          0.166667   
n9d39dea58c9e3cf              0.50           0.666667          0.500000   

                  target_thomas_20  
id                                  
n559bd06a8861222          0.333333  
n9d39dea58c9e3cf          0.500000

In [ ]:
num_dataf.head()

era data_type  feature_dichasial_hammier_spawner  \
id                                                                    
n559bd06a8861222  0297     train                               0.25   
n9d39dea58c9e3cf  0003     train                               0.75   
nb64f06d3a9fc9f1  0472     train                               1.00   
n1927b4862500882  0265     train                               0.00   
nc3234b6eeacd6b7  0299     train                               0.75   

                  feature_rheumy_epistemic_prancer  \
id                                                   
n559bd06a8861222                              0.75   
n9d39dea58c9e3cf                              0.50   
nb64f06d3a9fc9f1                              1.00   
n1927b4862500882                              0.00   
nc3234b6eeacd6b7                              0.25   

                  feature_pert_performative_hormuz  \
id                                                   
n559bd06a8861222                              0.25   
n9d39dea58c9e3cf                              0.75   
nb64f06d3a9fc9f1                              1.00   
n1927b4862500882                              0.25   
nc3234b6eeacd6b7                              0.00   

                  feature_hillier_unpitied_theobromine  \
id                                                       
n559bd06a8861222                                  0.75   
n9d39dea58c9e3cf                                  1.00   
nb64f06d3a9fc9f1                                  0.50   
n1927b4862500882                                  0.00   
nc3234b6eeacd6b7                                  0.75   

                  feature_perigean_bewitching_thruster  \
id                                                       
n559bd06a8861222                                  0.25   
n9d39dea58c9e3cf                                  0.50   
nb64f06d3a9fc9f1                                  0.00   
n1927b4862500882                                  1.00   
nc3234b6eeacd6b7                                  1.00   

                  feature_renegade_undomestic_milord  \
id                                                     
n559bd06a8861222                                0.50   
n9d39dea58c9e3cf                                0.25   
nb64f06d3a9fc9f1                                1.00   
n1927b4862500882                                0.00   
nc3234b6eeacd6b7                                0.25   

                  feature_koranic_rude_corf  \
id                                            
n559bd06a8861222                       1.00   
n9d39dea58c9e3cf                       0.50   
nb64f06d3a9fc9f1                       0.25   
n1927b4862500882                       0.00   
nc3234b6eeacd6b7                       0.00   

                  feature_demisable_expiring_millepede  ...  target_paul_20  \
id                                                      ...                   
n559bd06a8861222                                  0.25  ...            0.00   
n9d39dea58c9e3cf                                  0.00  ...            0.50   
nb64f06d3a9fc9f1                                  0.50  ...            0.00   
n1927b4862500882                                  0.00  ...            0.75   
nc3234b6eeacd6b7                                  0.00  ...            0.25   

                  target_paul_60  target_george_20  target_george_60  \
id                                                                     
n559bd06a8861222            0.50              0.25              0.50   
n9d39dea58c9e3cf            0.75              0.50              0.50   
nb64f06d3a9fc9f1            0.25              0.50              0.50   
n1927b4862500882            0.75              0.50              0.75   
nc3234b6eeacd6b7            0.50              0.50              0.50   

                  target_william_20  target_william_60  target_arthur_20  \
id                                                                         
n559bd06a886122

`.get_era_batch` will return a `tf.Tensor` or `np.array` with feature data and target data for one or more eras. Convenient for creating neural network DataGenerators.
Define `convert_to_tf` to return a `tf.Tensor` object. Else a `np.array` will be returned.

In [ ]:
X_era, y_era = num_dataf.get_era_batch(['0297'], convert_to_tf=False)
X_era

2023-03-23 18:06:22.672389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 18:06:22.918932: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-23 18:06:24.031738: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/miniconda3/envs/classic_prod/lib/
2023-03-23 18:06:24.031878: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plu

<tf.Tensor: shape=(1, 1050), dtype=float32, numpy=array([[0.25, 0.75, 0.25, ..., 0.  , 1.  , 0.25]], dtype=float32)>

For people training autoencoders + MLP you can get a target that contains 3 elements: features, targets and targets. Just define `aemlp_batch=True`.
More info on this setup: [AutoEncoder and multitask MLP on new dataset forum post](https://forum.numer.ai/t/autoencoder-and-multitask-mlp-on-new-dataset-from-kaggle-jane-street/4338).

In [ ]:
# _, y_era_aemlp = num_dataf.get_era_batch(['0297'], convert_to_tf=True, aemlp_batch=True)
# y_era_aemlp

[<tf.Tensor: shape=(1, 1050), dtype=float32, numpy=array([[0.25, 0.75, 0.25, ..., 0.  , 1.  , 0.25]], dtype=float32)>,
 <tf.Tensor: shape=(1, 21), dtype=float32, numpy=
 array([[0.25      , 0.25      , 0.5       , 0.        , 0.5       ,
         0.5       , 0.5       , 0.25      , 0.5       , 0.5       ,
         0.5       , 0.        , 0.5       , 0.25      , 0.5       ,
         0.        , 0.5       , 0.16666667, 0.5       , 0.33333334,
         0.5       ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 21), dtype=float32, numpy=
 array([[0.25      , 0.25      , 0.5       , 0.        , 0.5       ,
         0.5       , 0.5       , 0.25      , 0.5       , 0.5       ,
         0.5       , 0.        , 0.5       , 0.25      , 0.5       ,
         0.        , 0.5       , 0.16666667, 0.5       , 0.33333334,
         0.5       ]], dtype=float32)>]

`.aux_cols` denotes all columns that are not features, targets or prediction columns.

In [ ]:
num_dataf.aux_cols

['era', 'data_type']

In [ ]:
num_dataf.get_aux_data.head(2)

era data_type
id                              
n559bd06a8861222  0297     train
n9d39dea58c9e3cf  0003     train

In [ ]:
num_dataf['prediction_1'] = np.random.uniform(size=len(num_dataf))
num_dataf['prediction_2'] = np.random.uniform(size=len(num_dataf))

To track new columns like prediction columns, make sure to initialize a new `NumerFrame`. Prediction columns can easily be retrieved with `.get_prediction_data` and `get_prediction_aux_data` if you want to also get columns like `era` and `data_type`. This can be handy for ensembling and submission use cases.

In [ ]:
num_dataf = NumerFrame(num_dataf)

In [ ]:
num_dataf.get_prediction_data.head(2)

prediction_1  prediction_2
id                                          
n559bd06a8861222      0.087993      0.729183
n9d39dea58c9e3cf      0.238604      0.384513

In [ ]:
num_dataf.get_prediction_aux_data.head(2)

prediction_1  prediction_2   era data_type
id                                                          
n559bd06a8861222      0.087993      0.729183  0297     train
n9d39dea58c9e3cf      0.238604      0.384513  0003     train

In [ ]:
num_dataf.meta

{'era_col': 'era'}

Because `NumerFrame` inherits from `pd.DataFrame` you still have all functionality of a normal DataFrame at your disposal, like copying.

In [ ]:
dataf2 = num_dataf.copy()
assert dataf2.equals(num_dataf)

`NumerFrame` dynamically tracks which feature, target, aux and prediction columns there are when initialized. For example, here we add a new prediction column. Upon initialization the column will be contained in `prediction_cols`. Prediction columns are all column names that start with `prediction`.

In [ ]:
num_dataf.loc[:, "prediction_test_1"] = np.random.uniform(size=len(num_dataf))
new_dataset = NumerFrame(num_dataf)
assert "prediction_test_1" in new_dataset.prediction_cols

Arbitrary columns van be retrieved with `.get_column_selection`. The input argument can be either a string or a list with column names.

In [ ]:
selection1 = num_dataf.get_column_selection("era")
selection1.head(2)

era
id                    
n559bd06a8861222  0297
n9d39dea58c9e3cf  0003

In [ ]:
selection2 = num_dataf.get_column_selection(["era", "prediction_test_1"])
selection2.head(2)

era  prediction_test_1
id                                       
n559bd06a8861222  0297           0.952676
n9d39dea58c9e3cf  0003           0.616081

In [ ]:
#| include: false
for sel in [selection1, selection2]:
    assert isinstance(sel, NumerFrame)

For convenience we can get a feature, target pair with one method. If `multi_target=True` all columns where the column name starts with `target` will be retrieved.

In [ ]:
features, single_target = num_dataf.get_feature_target_pair(multi_target=False)
features.head(2)

feature_dichasial_hammier_spawner  \
id                                                    
n559bd06a8861222                               0.25   
n9d39dea58c9e3cf                               0.75   

                  feature_rheumy_epistemic_prancer  \
id                                                   
n559bd06a8861222                              0.75   
n9d39dea58c9e3cf                              0.50   

                  feature_pert_performative_hormuz  \
id                                                   
n559bd06a8861222                              0.25   
n9d39dea58c9e3cf                              0.75   

                  feature_hillier_unpitied_theobromine  \
id                                                       
n559bd06a8861222                                  0.75   
n9d39dea58c9e3cf                                  1.00   

                  feature_perigean_bewitching_thruster  \
id                                                       
n559bd06a8861222                                  0.25   
n9d39dea58c9e3cf                                  0.50   

                  feature_renegade_undomestic_milord  \
id                                                     
n559bd06a8861222                                0.50   
n9d39dea58c9e3cf                                0.25   

                  feature_koranic_rude_corf  \
id                                            
n559bd06a8861222                        1.0   
n9d39dea58c9e3cf                        0.5   

                  feature_demisable_expiring_millepede  \
id                                                       
n559bd06a8861222                                  0.25   
n9d39dea58c9e3cf                                  0.00   

                  feature_unscheduled_malignant_shingling  \
id                                                          
n559bd06a8861222                                     0.25   
n9d39dea58c9e3cf                                     1.00   

                  feature_clawed_unwept_adaptability  ...  \
id                                                    ...   
n559bd06a8861222                                0.75  ...   
n9d39dea58c9e3cf                                0.25  ...   

                  feature_unpruned_pedagoguish_inkblot  \
id                                                       
n559bd06a8861222                                  0.75   
n9d39dea58c9e3cf                                  1.00   

                  feature_forworn_hask_haet  \
id                                            
n559bd06a8861222                        0.0   
n9d39dea58c9e3cf                        1.0   

                  feature_drawable_exhortative_dispersant  \
id                                                          
n559bd06a8861222                                     1.00   
n9d39dea58c9e3cf                                     0.25   

                  feature_metabolic_minded_armorist  \
id                                                    
n559bd06a8861222                                0.0   
n9d39dea58c9e3cf                                0.5   

                  feature_investigatory_inerasable_circumvallation  \
id                                                                   
n559bd06a8861222                                               0.0   
n9d39dea58c9e3cf                                               0.0   

                  feature_centroclinal_incentive_lancelet  \
id                                                          
n559bd06a8861222                                     0.25   
n9d39dea58c9e3cf                                     0.25   

                  feature_unemotional_quietistic_chirper  \
id                                                         
n559bd06a8861222                                    0.00   
n9d39dea58c9e3cf                                    0.75   

                  feature_behaviorist_microbiological_farina  \
id                                       

In [ ]:
single_target.head(2)

target
id                      
n559bd06a8861222    0.25
n9d39dea58c9e3cf    0.50

-----------------------------------------------